In [50]:
# Import necessary libraries
import requests
import re
import pandas as pd

In [51]:
#creating a session object
session=requests.session()

In [52]:
#defining url
url='https://www.lenskart.com/stores'
#Send an HTTP GET request to the URL using the session
response=session.get(url)
print(response)

<Response [200]>


In [53]:
#  Write the HTML content to a file
with open('storelocator.html', 'w', encoding='utf-8') as file:
    # 'response.text' contains the HTML content of the response
    file.write(response.text)

In [54]:
#regular expressions to extract store locations from HTML content
store_locations = re.findall('class="undefined".*?><.*?>(.*?)<', response.text, re.S)
print(store_locations)

['Andhra Pradesh', 'Andmaan nicobar', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Chennai', 'Chhattisgarh', 'Dadra and Nagar Haveli', 'Delhi', 'Goa', 'Gujarat', 'Gwalior', 'Haryana', 'Himachal Pradesh', 'Hyderabad', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Nagaland', 'New Delhi', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Tamil\xa0Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'Utter Pradesh', 'Uttrakhand', 'West Bengal']


In [55]:
#defining store url
store_url='https://www.lenskart.com/stores/location/'


In [18]:
# Initialize an empty list 
all_stores = []

# Iterate through each store_locations
for store_state in store_locations:
    # Clean the store state name by replacing spaces with hyphens and converting to lowercase
    location = store_state.replace(' ', '-').lower()
    
    # Build the URL for the specific location
    location_url = store_url + location
    
    # Send an HTTP GET request to the location URL
    location_response = session.get(location_url)
    
    # Use regular expressions to extract store information from the location response
    store_city_name = re.findall('class="StoreCard_name__mrTXJ">(.*?)<', location_response.text, re.S)
    store_address = re.findall('<a[^>]*\sclass="StoreCard_storeAddress__PfC_v null"[^>]*>(.*?)<\/a>', location_response.text, re.S)
    store_number = re.findall('class="StoreCard_wrapper__xhJ0A">.*?><a.*?>\s<.*?>(.*?)<', location_response.text, re.S)
    store_timings = re.findall('<div[^>]*\sclass="StoreCard_storeAddress__PfC_v null"[^>]*>.*?>(.*?)<', location_response.text, re.S)
    
    # Iterate through the extracted store information and create a dictionary for each store
    for city_name, address, number, timings in zip(store_city_name, store_address, store_number, store_timings):
        store_info = {
            'city_name': city_name.strip(),
            'address': address.strip(),
            'store_state': store_state.strip(),
            'number': number.strip(),
            'timings': timings.strip(),
        }
        # Append the store information dictionary to the list of all stores
        all_stores.append(store_info)

# Print the list of all stores
print("All Stores Information:")
print(all_stores)


[{'city_name': 'Vizianagaram', 'address': '20-20-103, Plot No103, Madhuvan Towers, Krishnarajapuram, Near Spencers, Vizianagaram, Andhra Pradesh, 535002', 'store_state': 'Andhra Pradesh', 'number': '9319374906', 'timings': '11:00 AM to 09:00 PM'}, {'city_name': 'Ramnagar', 'address': 'Shop No 10/4/16/2, Upper Ground Floor, Uplands Road, Asilmetta Junction, Ramnagar, Beside Hotel Meghalaya, Visakhapatnam, Andhra Pradesh, 530003', 'store_state': 'Andhra Pradesh', 'number': '7428890015', 'timings': '11:00 AM to 09:30 PM'}, {'city_name': 'Kakinada', 'address': '20-1-34, Subash Road, Opposite Super Bazar, Kakinada, Andhra Pradesh, 533001', 'store_state': 'Andhra Pradesh', 'number': '8447732252', 'timings': '11:00 AM to 09:00 PM'}, {'city_name': 'Eluru', 'address': '24B-1-46/1, Vijaya Vihar Centre, Vijaya Vihar Centre, Opposite Gv Mall, Eluru, Andhra Pradesh, 534002', 'store_state': 'Andhra Pradesh', 'number': '9319369743', 'timings': '11:00 AM to 09:00 PM'}, {'city_name': 'Shanthi Nagar', '

In [24]:
# Create a Pandas DataFrame
df = pd.DataFrame(all_stores)

# Specify the path for the Excel file where the DataFrame will be saved
excel_file_path = 'store_details.xlsx'

# Save the DataFrame to an Excel file without including the index column
df.to_excel(excel_file_path, index=False)
